In [1]:
import numpy as np
from pyteomics.mzml import read, iterfind
# import matplotlib.pyplot as plt


In [2]:
def save_mat2csv(fname, matrix, times=None, wls=None, unit=''):
    delimiter = ','
    times = np.arange(0, matrix.shape[0]) if times is None else times
    wls = np.arange(0, matrix.shape[1]) if wls is None else wls

    mat = np.hstack((times[:, None], matrix))
    buffer = f'unit: {unit} - Elution Time | m/z->'
    buffer += delimiter + delimiter.join(f"{num}" for num in wls) + '\n'
    buffer += '\n'.join(delimiter.join(f"{num}" for num in row) for row in mat)

    with open(fname, 'w', encoding='utf8') as f:
        f.write(buffer)

In [45]:
files = [
    '2Z_high_conc..mzML',
    '2ZRB_MeOH_545_irr.mzML',
    'MeO-PDPs.mzML'
]


In [51]:
def parse_mzml_file(filepath, mz_resolution=1):

    mzml_file = read(filepath)
    spectrumList_dict = iterfind(filepath, 'indexedmzML/mzML/run/spectrumList', read_schema=True, recursive=False).__next__()
    n_spectra = int(spectrumList_dict['count'])  # number of all spectra

    mzmin, mzmax = None, None
    mz_array = None
    mat = None
    times = None

    for i, sp in enumerate(mzml_file):

        if mat is None:
            mzmin = float(sp['scanList']['scan'][0]['scanWindowList']['scanWindow'][0]['scan window lower limit'])
            mzmax = float(sp['scanList']['scan'][0]['scanWindowList']['scanWindow'][0]['scan window upper limit'])

            mzmax = mzmax + mz_resolution - (mzmax - mzmin) % mz_resolution  # recalculate the maximum m/z value

            mz_array = np.linspace(mzmin, mzmax, int((mzmax - mzmin) / mz_resolution) + 1) # make sure to have evenly spaced integerers
            mat = np.zeros((n_spectra, mz_array.shape[0]))
            times = np.zeros(n_spectra)

        indexes = (sp['m/z array'] * mz_array.shape[0] / mzmax).astype(int)

        # find duplicated and integrate (just sum) them 
        indexes, indices, counts = np.unique(indexes, return_index=True, return_counts=True)
        intensities = sp['intensity array']
        integrated_intensities = intensities[indices]

        for j in range(indexes.shape[0]):
            if counts[j] < 2:
                continue

            idx = indices[j]
            integrated_intensities[j] = intensities[idx:idx + counts[j]].sum()

        try:
            times[i] = float(sp['scanList']['scan'][0]['scan start time'])
        except KeyError:
            pass
        mat[i, indexes] = integrated_intensities

    save_mat2csv(f'{filepath}.csv', mat, times, mz_array)


In [52]:
for file in files:
    parse_mzml_file(file, 2)

In [12]:
# spectrum = obj_file.__next__()
# plt.plot(spectrum['m/z array'], spectrum['intensity array'])
# spectrum['intensity array'].shape

array = np.asarray([1, 2, 2, 3, 4, 4, 6, 7])
vals, indices, counts = np.unique(array, return_index=True, return_counts=True)
vals, indices, counts


(array([1, 2, 3, 4, 6, 7]),
 array([0, 1, 3, 4, 6, 7], dtype=int64),
 array([1, 2, 1, 2, 1, 1], dtype=int64))